In [9]:
import pandas as pd

In [10]:
file_dict = {
    '2019 Election Results (1).xlsx': [2, 38],
    '2020 Election Results (1).xlsx': [2, 43],
    '2021 Election Results (1).xlsx': [2, 33],
    '2022 Election Results (1).xlsx': [2, 38]
}

In [26]:
def concat_excel_tabs(files:dict) -> tuple[pd.DataFrame, pd.DataFrame]:

    county_votes_df = pd.DataFrame()
    all_county_dfs = []
    reg_votes_df = pd.DataFrame()
    all_reg_votes_dfs = []
    name_map = {'Unnamed: 0': 'County', 'Unnamed: 1': 'Registered Voters'}

    for k, v in files.items():
        file_name = k
        tab_start = v[0]
        tab_end = v[1]+1
        #tab_end = 4 # for testing

        for tab in range(tab_start, tab_end):
            year = file_name[:4]
            scope = pd.read_excel(file_name, sheet_name=str(tab)).columns[0]
            df = pd.read_excel(file_name, sheet_name=str(tab), skiprows=1)
            df = df.iloc[1:, :].copy()
            df.rename(name_map, axis=1, inplace=True)
            df = df.melt(id_vars='County').copy()

            new_county_votes_df = df[df.variable != 'Registered Voters'].copy()
            new_county_votes_df['Year'] = year
            new_county_votes_df['Election'] = scope
            all_county_dfs.append(new_county_votes_df)

            new_reg_votes_df = df[df.variable == 'Registered Voters'].copy()
            new_reg_votes_df['Year'] = year
            new_reg_votes_df['Election'] = scope
            all_reg_votes_dfs.append(new_reg_votes_df)

    final_list_county_dfs = [county_votes_df] + all_county_dfs
    final_county_votes_df = pd.concat(final_list_county_dfs, axis=0)
    final_county_votes_df.columns = ['seat', 'candidate', 'votes', 'year', 'election']

    final_list_reg_votes_dfs = [reg_votes_df] + all_reg_votes_dfs
    final_reg_votes_df = pd.concat(final_list_reg_votes_dfs, axis=0)
    final_reg_votes_df.columns = ['seat', 'variable', 'total', 'year', 'election']

    return (final_county_votes_df, final_reg_votes_df)

In [27]:
county_votes, reg_votes = concat_excel_tabs(file_dict)

In [31]:
with pd.ExcelWriter('election_results_clean.xlsx') as writer:
    county_votes.to_excel(writer, sheet_name='election_results', index=False)
    reg_votes.to_excel(writer, sheet_name='registered_voters', index=False)